# Exp Q2: Hysteresis / Path Dependence

## 目的
相が「状態」なら、λを途中で変えた時に「戻らなさ」（履歴依存）が出る可能性を検証。
出れば一次転移様の証拠になり、PRE的に強い。

## 実験設計
- **Schedules**:
  1. Fixed（ベースライン）
  2. Quench: ordered→collapse（epoch 30で λ: 0.35→0.60）
  3. Reverse quench: collapse→ordered（epoch 30で λ: 0.60→0.35）
  4. Ramp up（線形で上昇）
  5. Ramp down（線形で下降）
- **η**: 0.4, 0.8
- **Seeds**: 5
- **Total**: 5 schedules × 2 η × 5 seeds = **50 runs**

## 推定時間
~50 runs × 9 min ≈ **7.5 時間**

In [ ]:
# ===== セットアップ =====
from google.colab import drive
drive.mount('/content/drive')

import os
import glob
from datetime import datetime

EXP_NAME = 'exp_Q2_hysteresis'
BASE_DIR = '/content/drive/MyDrive/dual-gradient-learning/Paper-A'

existing = glob.glob(f'{BASE_DIR}/{EXP_NAME}_*')
if existing:
    SAVE_DIR = sorted(existing)[-1]
    print(f'🔄 Resuming from: {SAVE_DIR}')
else:
    TIMESTAMP = datetime.now().strftime('%Y%m%d_%H%M%S')
    SAVE_DIR = f'{BASE_DIR}/{EXP_NAME}_{TIMESTAMP}'
    os.makedirs(SAVE_DIR, exist_ok=True)
    print(f'🆕 New experiment: {SAVE_DIR}')

os.makedirs(f'{SAVE_DIR}/figures', exist_ok=True)
print(f'Save directory: {SAVE_DIR}')

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils import parameters_to_vector
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet18
import numpy as np
import json
import time

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name()}')

In [ ]:
# ===== ResNet18 for CIFAR-10 =====
def get_resnet18():
    model = resnet18(weights=None, num_classes=10)
    model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
    model.maxpool = nn.Identity()
    return model

# ===== IndexedDataset =====
class IndexedDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
    def __getitem__(self, idx):
        img, label = self.dataset[idx]
        return img, label, idx
    def __len__(self):
        return len(self.dataset)

In [ ]:
# ===== λスケジュール定義 =====

class LambdaSchedule:
    """λのスケジュールを管理"""
    def __init__(self, schedule_type, eta, total_epochs=100):
        self.schedule_type = schedule_type
        self.eta = eta
        self.total_epochs = total_epochs
        
        # η依存のパラメータ設定
        if eta == 0.4:
            self.lam_ordered = 0.35
            self.lam_collapse = 0.60
            self.lam_critical = 0.48
        else:  # eta == 0.8
            self.lam_ordered = 0.35
            self.lam_collapse = 0.55
            self.lam_critical = 0.44
        
        self.quench_epoch = 30  # クエンチを行うエポック
    
    def get_lambda(self, epoch):
        if self.schedule_type == 'fixed_ordered':
            return self.lam_ordered
        
        elif self.schedule_type == 'fixed_collapse':
            return self.lam_collapse
        
        elif self.schedule_type == 'quench_to_collapse':
            # ordered → collapse at epoch 30
            if epoch < self.quench_epoch:
                return self.lam_ordered
            else:
                return self.lam_collapse
        
        elif self.schedule_type == 'quench_to_ordered':
            # collapse → ordered at epoch 30
            if epoch < self.quench_epoch:
                return self.lam_collapse
            else:
                return self.lam_ordered
        
        elif self.schedule_type == 'ramp_up':
            # 線形に上昇: ordered → collapse
            progress = epoch / self.total_epochs
            return self.lam_ordered + progress * (self.lam_collapse - self.lam_ordered)
        
        elif self.schedule_type == 'ramp_down':
            # 線形に下降: collapse → ordered
            progress = epoch / self.total_epochs
            return self.lam_collapse - progress * (self.lam_collapse - self.lam_ordered)
        
        else:
            raise ValueError(f'Unknown schedule: {self.schedule_type}')
    
    def describe(self):
        return f'{self.schedule_type} (η={self.eta})'

# スケジュールをテスト表示
print('Lambda schedules (η=0.4):')
for stype in ['fixed_ordered', 'fixed_collapse', 'quench_to_collapse', 'quench_to_ordered', 'ramp_up', 'ramp_down']:
    sched = LambdaSchedule(stype, 0.4)
    lambdas = [sched.get_lambda(e) for e in [0, 29, 30, 50, 99]]
    print(f'  {stype}: {lambdas}')

In [ ]:
# ===== 実験パラメータ =====
BATCH_SIZE = 256
NUM_WORKERS = 4
EPOCHS = 100
LR = 0.1
K = 16

SCHEDULE_TYPES = [
    'fixed_ordered',      # ベースライン（ordered）
    'fixed_collapse',     # ベースライン（collapse）
    'quench_to_collapse', # ordered → collapse
    'quench_to_ordered',  # collapse → ordered
    'ramp_up',            # 線形上昇
    'ramp_down'           # 線形下降
]

NOISE_RATES = [0.4, 0.8]
SEEDS = list(range(5))

experiments = []
for stype in SCHEDULE_TYPES:
    for eta in NOISE_RATES:
        for seed in SEEDS:
            experiments.append({
                'schedule_type': stype,
                'noise_rate': eta,
                'seed': seed
            })

print(f'Schedule types: {SCHEDULE_TYPES}')
print(f'Noise rates: {NOISE_RATES}')
print(f'Total experiments: {len(experiments)}')
print(f'Estimated time: {len(experiments) * 9 / 60:.1f} hours')

# Save config
config = {
    'experiment': EXP_NAME,
    'description': 'Hysteresis/path dependence: λ schedule effects',
    'parameters': {
        'schedule_types': SCHEDULE_TYPES,
        'noise_rates': NOISE_RATES,
        'seeds': SEEDS,
        'epochs': EPOCHS,
        'batch_size': BATCH_SIZE,
        'lr': LR,
        'K': K,
        'quench_epoch': 30
    },
    'total_runs': len(experiments)
}
with open(f'{SAVE_DIR}/{EXP_NAME}_config.json', 'w') as f:
    json.dump(config, f, indent=2)

In [ ]:
# ===== ユーティリティ関数 =====
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)

def load_cifar10():
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))
    ])
    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))
    ])
    trainset = torchvision.datasets.CIFAR10('./data', train=True, download=True, transform=transform_train)
    testset = torchvision.datasets.CIFAR10('./data', train=False, download=True, transform=transform_test)
    return trainset, testset

def inject_label_noise(labels, noise_rate, seed):
    np.random.seed(seed)
    labels = np.array(labels)
    n_samples = len(labels)
    n_noisy = int(noise_rate * n_samples)
    noisy_idx = np.random.choice(n_samples, n_noisy, replace=False)
    noisy_labels = labels.copy()
    for idx in noisy_idx:
        choices = [l for l in range(10) if l != labels[idx]]
        noisy_labels[idx] = np.random.choice(choices)
    return noisy_labels

def get_data_loaders(trainset, testset):
    train_loader = DataLoader(IndexedDataset(trainset), batch_size=BATCH_SIZE,
                              shuffle=True, num_workers=NUM_WORKERS, pin_memory=True)
    test_loader = DataLoader(testset, batch_size=BATCH_SIZE,
                             shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)
    return train_loader, test_loader

def evaluate(model, test_loader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    return correct / total

In [ ]:
# ===== Dual-Gradient Learning with λ Schedule =====
def train_dual_gradient_scheduled(model, train_loader, test_loader, clean_labels, noisy_labels, schedule):
    optimizer = optim.SGD(model.parameters(), lr=LR, momentum=0.9, weight_decay=5e-4)
    scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[50, 75], gamma=0.1)
    criterion = nn.CrossEntropyLoss()
    
    clean_labels_tensor = torch.tensor(clean_labels, device=device)
    noisy_labels_tensor = torch.tensor(noisy_labels, device=device)
    
    cached_g_value = None
    global_step = 0
    
    # 詳細な履歴を記録
    history = {
        'epoch': [],
        'lambda': [],
        'test_error': [],
        'avg_cos': []
    }
    
    for epoch in range(EPOCHS):
        # エポックごとのλを取得
        current_lambda = schedule.get_lambda(epoch)
        
        model.train()
        epoch_cos = []
        
        for inputs, _, indices in train_loader:
            inputs = inputs.to(device, non_blocking=True)
            indices = indices.to(device, non_blocking=True)
            batch_noisy = noisy_labels_tensor[indices]
            batch_clean = clean_labels_tensor[indices]
            
            # Structure gradient
            optimizer.zero_grad()
            outputs = model(inputs)
            loss_struct = criterion(outputs, batch_noisy)
            loss_struct.backward(retain_graph=True)
            g_struct = parameters_to_vector([p.grad for p in model.parameters()]).clone()
            
            # Value gradient
            if global_step % K == 0 or cached_g_value is None:
                optimizer.zero_grad()
                outputs = model(inputs)
                loss_value = criterion(outputs, batch_clean)
                loss_value.backward()
                cached_g_value = parameters_to_vector([p.grad for p in model.parameters()]).clone()
            
            # Mix gradients with current λ
            g_struct_norm = g_struct / (g_struct.norm() + 1e-12)
            g_value_norm = cached_g_value / (cached_g_value.norm() + 1e-12)
            
            cos_sim = (g_struct_norm @ g_value_norm).item()
            epoch_cos.append(cos_sim)
            
            g_mix = (1 - current_lambda) * g_struct_norm + current_lambda * g_value_norm
            
            # Apply mixed gradient
            optimizer.zero_grad()
            idx = 0
            for p in model.parameters():
                numel = p.numel()
                p.grad = g_mix[idx:idx+numel].view(p.shape).clone()
                idx += numel
            optimizer.step()
            global_step += 1
        
        scheduler.step()
        
        # 毎エポック記録（ヒステリシス解析のため）
        acc = evaluate(model, test_loader)
        history['epoch'].append(epoch)
        history['lambda'].append(current_lambda)
        history['test_error'].append(1 - acc)
        history['avg_cos'].append(np.mean(epoch_cos))
    
    final_acc = evaluate(model, test_loader)
    
    return final_acc, history

In [ ]:
# ===== データ準備 =====
trainset, testset = load_cifar10()
clean_labels = np.array(trainset.targets)
train_loader, test_loader = get_data_loaders(trainset, testset)

print('Data prepared')

# GPU warmup
warmup_model = get_resnet18().to(device)
for _ in range(20):
    _ = warmup_model(torch.randn(BATCH_SIZE, 3, 32, 32, device=device))
del warmup_model
torch.cuda.empty_cache()
print('GPU warmed up')

In [ ]:
# ===== メイン実験ループ =====
results = []
checkpoint_file = f'{SAVE_DIR}/{EXP_NAME}_checkpoint.json'
completed = set()

if os.path.exists(checkpoint_file):
    with open(checkpoint_file, 'r') as f:
        results = json.load(f)
    for r in results:
        completed.add((r['schedule_type'], r['noise_rate'], r['seed']))
    print(f'Checkpoint loaded: {len(results)} runs completed')

total = len(experiments)

for exp in experiments:
    stype = exp['schedule_type']
    eta = exp['noise_rate']
    seed = exp['seed']
    
    if (stype, eta, seed) in completed:
        continue
    
    run_num = len(completed) + 1
    print(f'\n[{run_num}/{total}] {stype} η={eta} seed={seed}')
    
    set_seed(seed)
    noisy_labels = inject_label_noise(clean_labels, eta, seed)
    
    model = get_resnet18().to(device)
    schedule = LambdaSchedule(stype, eta, EPOCHS)
    
    start_time = time.time()
    final_acc, history = train_dual_gradient_scheduled(
        model, train_loader, test_loader, clean_labels, noisy_labels, schedule
    )
    elapsed = time.time() - start_time
    
    result = {
        'experiment_id': f'Q2-{run_num:03d}',
        'experiment': EXP_NAME,
        'schedule_type': stype,
        'noise_rate': eta,
        'seed': seed,
        'final_acc': final_acc,
        'final_error': 1 - final_acc,
        'time_seconds': elapsed,
        'history': history
    }
    results.append(result)
    completed.add((stype, eta, seed))
    
    status = '✅' if final_acc > 0.85 else ('⚠️ COLLAPSE' if final_acc < 0.5 else '')
    print(f'  Final error: {1-final_acc:.4f} | Time: {elapsed/60:.1f} min {status}')
    
    with open(checkpoint_file, 'w') as f:
        json.dump(results, f, indent=2)
    
    remaining = total - run_num
    eta_hours = remaining * elapsed / 3600
    print(f'  Progress: {run_num}/{total} | ETA: {eta_hours:.1f} hours')
    
    del model
    torch.cuda.empty_cache()

print('\n' + '='*70)
print('ALL EXPERIMENTS COMPLETED!')
print('='*70)

In [ ]:
# ===== 結果保存 =====
import pandas as pd

with open(f'{SAVE_DIR}/{EXP_NAME}_results.json', 'w') as f:
    json.dump(results, f, indent=2)

results_flat = [{k: v for k, v in r.items() if k != 'history'} for r in results]
df = pd.DataFrame(results_flat)
df.to_csv(f'{SAVE_DIR}/{EXP_NAME}_results.csv', index=False)

print(f'Results saved to {SAVE_DIR}')

In [ ]:
# ===== ヒステリシス分析 =====
import matplotlib.pyplot as plt

print('='*70)
print('HYSTERESIS ANALYSIS')
print('='*70)

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

schedule_colors = {
    'fixed_ordered': '#2ecc71',
    'fixed_collapse': '#e74c3c',
    'quench_to_collapse': '#9b59b6',
    'quench_to_ordered': '#3498db',
    'ramp_up': '#f39c12',
    'ramp_down': '#1abc9c'
}

for i, eta in enumerate([0.4, 0.8]):
    # Left: Learning curves
    ax = axes[i, 0]
    for stype in SCHEDULE_TYPES:
        # 代表的なseed=0の曲線を描画
        for r in results:
            if r['schedule_type'] == stype and r['noise_rate'] == eta and r['seed'] == 0:
                history = r['history']
                ax.plot(history['epoch'], history['test_error'],
                       color=schedule_colors[stype], label=stype, linewidth=1.5, alpha=0.8)
                break
    
    ax.axvline(x=30, color='gray', linestyle='--', alpha=0.5, label='Quench point')
    ax.set_xlabel('Epoch', fontsize=12)
    ax.set_ylabel('Test Error', fontsize=12)
    ax.set_title(f'η={eta}: Learning Curves by Schedule', fontsize=12)
    ax.legend(loc='upper right', fontsize=8)
    ax.grid(True, alpha=0.3)
    
    # Right: Final error comparison
    ax = axes[i, 1]
    df_eta = df[df['noise_rate'] == eta]
    df_agg = df_eta.groupby('schedule_type')['final_error'].agg(['mean', 'std']).reset_index()
    
    x_pos = range(len(df_agg))
    bars = ax.bar(x_pos, df_agg['mean'], yerr=df_agg['std'], capsize=4,
                  color=[schedule_colors.get(s, 'gray') for s in df_agg['schedule_type']])
    ax.set_xticks(x_pos)
    ax.set_xticklabels(df_agg['schedule_type'], rotation=45, ha='right', fontsize=9)
    ax.set_ylabel('Final Error', fontsize=12)
    ax.set_title(f'η={eta}: Final Error by Schedule', fontsize=12)
    ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig(f'{SAVE_DIR}/figures/{EXP_NAME}_hysteresis.png', dpi=150, bbox_inches='tight')
plt.show()

# ヒステリシス判定
print('\n--- Hysteresis Detection ---')
for eta in [0.4, 0.8]:
    print(f'\nη={eta}:')
    df_eta = df[df['noise_rate'] == eta]
    
    # fixed_orderedとfixed_collapseのベースライン
    baseline_ordered = df_eta[df_eta['schedule_type'] == 'fixed_ordered']['final_error'].mean()
    baseline_collapse = df_eta[df_eta['schedule_type'] == 'fixed_collapse']['final_error'].mean()
    
    print(f'  Baselines: ordered={baseline_ordered:.4f}, collapse={baseline_collapse:.4f}')
    
    # クエンチ後の最終状態
    quench_to_collapse = df_eta[df_eta['schedule_type'] == 'quench_to_collapse']['final_error'].mean()
    quench_to_ordered = df_eta[df_eta['schedule_type'] == 'quench_to_ordered']['final_error'].mean()
    
    print(f'  Quench to collapse: {quench_to_collapse:.4f} (expect ~{baseline_collapse:.4f})')
    print(f'  Quench to ordered: {quench_to_ordered:.4f} (expect ~{baseline_ordered:.4f})')
    
    # ヒステリシス判定
    hysteresis_collapse = abs(quench_to_collapse - baseline_collapse)
    hysteresis_ordered = abs(quench_to_ordered - baseline_ordered)
    
    if hysteresis_collapse > 0.05 or hysteresis_ordered > 0.05:
        print(f'  ⚡ HYSTERESIS DETECTED! Gaps: {hysteresis_collapse:.4f}, {hysteresis_ordered:.4f}')
    else:
        print(f'  ✅ No significant hysteresis (gaps: {hysteresis_collapse:.4f}, {hysteresis_ordered:.4f})')